In [1]:
# --- 1. Setup and Imports ---

import sys
import subprocess
import importlib

# Install necessary packages using the robust subprocess method
print("Installing required packages...")
command = [
    sys.executable, '-m', 'pip', 'install', '-q',
    'opencv-python-headless', 'segmentation-models-pytorch', 'timm', 'albumentations'
]
result = subprocess.run(command, capture_output=True, text=True)

if result.returncode == 0:
    print("\u2705 Packages installed successfully.")
    importlib.invalidate_caches()
else:
    print("\u274c Package installation failed.")
    print(result.stderr)

Installing required packages...


✅ Packages installed successfully.


In [17]:
# --- 2. Configuration and Imports ---

import os
import gc
import glob
import random
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast

import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

class CFG:
    # Paths
    TRAIN_PATH = 'train/'
    TEST_PATH = 'test/'
    
    # Data settings from EDA
    Z_START = 20
    Z_END = 44  # Use slices from 20 to 43, inclusive (24 slices)
    
    # Model Input
    IN_CHANS = Z_END - Z_START + 1 # 24 slices + 1 IR channel
    TILE_SIZE = 320
    STRIDE = TILE_SIZE // 2

    # Training settings
    VALID_FRAGMENTS = ['2'] # Start with fold 1 (validate on fragment 2)
    BATCH_SIZE = 32
    EPOCHS = 20
    LR = 3e-4
    WEIGHT_DECAY = 1e-4
    PATIENCE = 5 # Early stopping patience
    
    # Model architecture
    MODEL_NAME = 'FPN'
    BACKBONE = 'timm-efficientnet-b4' # Changed from tf_efficientnetv2_s which is not supported
    
    # Environment
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    SEED = 42

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CFG.SEED)
print(f"Using device: {CFG.DEVICE}")
print(f"Input channels: {CFG.IN_CHANS}")

Using device: cuda
Input channels: 25


In [3]:
# --- 3. Data Loading and Preprocessing ---

def read_data(fragment_id, is_test=False):
    """Reads the data for a single fragment, including slices, mask, and IR image."""
    if is_test:
        path = os.path.join(CFG.TEST_PATH, fragment_id)
    else:
        path = os.path.join(CFG.TRAIN_PATH, fragment_id)
    
    # Load mask (ROI and ink labels)
    mask_path = os.path.join(path, 'inklabels.png')
    if os.path.exists(mask_path):
        ink_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    else: # For test set, there are no ink labels
        # Create a placeholder mask of the same shape as the ROI mask
        roi_mask_path = os.path.join(path, 'mask.png')
        roi_mask_shape = cv2.imread(roi_mask_path, cv2.IMREAD_GRAYSCALE).shape
        ink_mask = np.zeros(roi_mask_shape, dtype=np.uint8)
        
    roi_mask = cv2.imread(os.path.join(path, 'mask.png'), cv2.IMREAD_GRAYSCALE)
    
    # Load IR image
    ir_image = cv2.imread(os.path.join(path, 'ir.png'), cv2.IMREAD_GRAYSCALE)
    
    # Load TIF slices
    images = []
    slice_path_pattern = os.path.join(path, 'surface_volume', '*.tif')
    slice_paths = sorted(glob.glob(slice_path_pattern))
    for i in range(CFG.Z_START, CFG.Z_END):
        img = cv2.imread(slice_paths[i], cv2.IMREAD_UNCHANGED)
        images.append(img)
    images = np.stack(images, axis=-1)
    
    return images, ink_mask, roi_mask, ir_image

def get_transforms(is_train):
    """Returns the Albumentations transforms for training or validation."""
    if is_train:
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            ToTensorV2(transpose_mask=True),
        ])
    else:
        return A.Compose([
            ToTensorV2(transpose_mask=True),
        ])

print("Data loading functions and transforms defined.")

Data loading functions and transforms defined.


In [13]:
# --- 4. Custom PyTorch Dataset ---

class VesuviusDataset(Dataset):
    def __init__(self, fragment_ids, is_train, transforms=None):
        self.fragment_ids = fragment_ids
        self.is_train = is_train
        self.transforms = transforms
        
        self.images = []
        self.ink_masks = []
        self.ir_images = []
        self.coords = []

        for fragment_id in self.fragment_ids:
            images, ink_mask, roi_mask, ir_image = read_data(fragment_id, is_test=False)
            
            # Crop to ROI bounding box to save memory and computation
            x_indices, y_indices = np.where(roi_mask > 0)
            x_min, x_max = np.min(x_indices), np.max(x_indices)
            y_min, y_max = np.min(y_indices), np.max(y_indices)
            
            images = images[x_min:x_max, y_min:y_max]
            ink_mask = ink_mask[x_min:x_max, y_min:y_max]
            ir_image = ir_image[x_min:x_max, y_min:y_max]
            roi_mask = roi_mask[x_min:x_max, y_min:y_max]
            
            # Generate tile coordinates
            img_h, img_w = roi_mask.shape
            ink_coords_fragment = []
            no_ink_coords_fragment = []
            
            for y in range(0, img_h - CFG.TILE_SIZE + 1, CFG.STRIDE):
                for x in range(0, img_w - CFG.TILE_SIZE + 1, CFG.STRIDE):
                    # BUG FIX: Check percentage of tile within ROI
                    if (roi_mask[y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE] > 0).mean() > 0.5:
                        # Check if the tile contains ink
                        if ink_mask[y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE].sum() > 0:
                            ink_coords_fragment.append((len(self.images), y, x))
                        else:
                            no_ink_coords_fragment.append((len(self.images), y, x))
            
            self.images.append(images)
            self.ink_masks.append(ink_mask)
            self.ir_images.append(ir_image)
            
            # For balanced sampling
            if self.is_train:
                # Balance the dataset by taking all ink tiles and an equal number of no-ink tiles (if available)
                self.coords.extend(ink_coords_fragment)
                num_no_ink_samples = min(len(ink_coords_fragment), len(no_ink_coords_fragment))
                if len(no_ink_coords_fragment) > 0:
                    self.coords.extend(random.sample(no_ink_coords_fragment, num_no_ink_samples))
            else:
                # For validation, use all valid tiles
                self.coords.extend(ink_coords_fragment)
                self.coords.extend(no_ink_coords_fragment)

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, idx):
        fragment_idx, y, x = self.coords[idx]
        
        # Get tiles
        tif_tile = self.images[fragment_idx][y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE]
        ir_tile = self.ir_images[fragment_idx][y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE]
        mask_tile = self.ink_masks[fragment_idx][y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE]
        
        # BUG FIX: Normalize TIF (16-bit) and IR (8-bit) separately
        tif_tile = tif_tile.astype(np.float32) / 65535.0
        ir_tile = ir_tile.astype(np.float32) / 255.0
        
        # Combine slices and IR image
        ir_tile = np.expand_dims(ir_tile, axis=-1)
        img_tile = np.concatenate([tif_tile, ir_tile], axis=-1)
        
        # Normalize mask
        mask_tile = mask_tile.astype(np.float32) / 255.0
        
        if self.transforms:
            augmented = self.transforms(image=img_tile, mask=mask_tile)
            img_tile = augmented['image']
            mask_tile = augmented['mask']
        
        return img_tile, mask_tile.unsqueeze(0)

print("Custom Dataset class defined (with bug fixes).")

Custom Dataset class defined (with bug fixes).


In [14]:
# --- 5. Create Datasets and DataLoaders ---

print("Creating datasets and dataloaders...")

# Define train and validation fragments for the first fold
all_fragments = ['1', '2']
train_fragments = [f for f in all_fragments if f not in CFG.VALID_FRAGMENTS]

print(f"Training on fragments: {train_fragments}")
print(f"Validating on fragments: {CFG.VALID_FRAGMENTS}")

# Create datasets
train_dataset = VesuviusDataset(fragment_ids=train_fragments, is_train=True, transforms=get_transforms(is_train=True))
valid_dataset = VesuviusDataset(fragment_ids=CFG.VALID_FRAGMENTS, is_train=False, transforms=get_transforms(is_train=False))

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(valid_dataset)}")

# Verify a batch from the train_loader
try:
    images, masks = next(iter(train_loader))
    print(f"\n--- Batch Verification ---")
    print(f"Images batch shape: {images.shape}")
    print(f"Masks batch shape: {masks.shape}")
    print(f"Images dtype: {images.dtype}")
    print(f"Masks dtype: {masks.dtype}")
except Exception as e:
    print(f"\nError during batch verification: {e}")

Creating datasets and dataloaders...
Training on fragments: ['1']
Validating on fragments: ['2']


Train dataset size: 1130
Validation dataset size: 3817



--- Batch Verification ---
Images batch shape: torch.Size([32, 25, 320, 320])
Masks batch shape: torch.Size([32, 1, 320, 320])
Images dtype: torch.float32
Masks dtype: torch.float32


In [10]:
# --- 6. Model, Loss, and Optimizer ---

print("Defining model, loss function, and optimizer...")

# Define the model using segmentation-models-pytorch
model = smp.create_model(
    arch=CFG.MODEL_NAME,
    encoder_name=CFG.BACKBONE,
    in_channels=CFG.IN_CHANS,
    classes=1, # Binary segmentation
    activation=None, # We'll use BCEWithLogitsLoss which includes sigmoid
)
model.to(CFG.DEVICE)

# Define loss function (combination of BCE and Dice)
class ComboLoss(nn.Module):
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super(ComboLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.dice_loss = smp.losses.DiceLoss(mode='binary')
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, outputs, targets):
        bce = self.bce_loss(outputs, targets)
        dice = self.dice_loss(outputs, targets)
        return self.bce_weight * bce + self.dice_weight * dice

loss_fn = ComboLoss()

# Define optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=CFG.LR, weight_decay=CFG.WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.EPOCHS, eta_min=1e-6)

print(f"Model: {CFG.MODEL_NAME} with backbone {CFG.BACKBONE}")
print("Loss: ComboLoss (BCE + Dice)")
print("Optimizer: AdamW")

Defining model, loss function, and optimizer...


config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

Model: FPN with backbone timm-efficientnet-b4
Loss: ComboLoss (BCE + Dice)
Optimizer: AdamW


In [16]:
# --- 7. Training and Validation Helper Functions ---

def calc_fbeta(tp, fp, fn, beta=0.5):
    """Calculates F-beta score from TP, FP, FN."""
    if tp + fp == 0:
        precision = 0.0
    else:
        precision = tp / (tp + fp)
        
    if tp + fn == 0:
        recall = 0.0
    else:
        recall = tp / (tp + fn)

    if (beta**2 * precision) + recall == 0:
        fbeta = 0.0
    else:
        fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
        
    return fbeta, precision, recall

def train_one_epoch(model, optimizer, scheduler, dataloader, device, scaler):
    """Runs a single training epoch."""
    model.train()
    epoch_loss = 0
    
    for images, masks in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad()
        
        with autocast():
            outputs = model(images)
            loss = loss_fn(outputs, masks)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        epoch_loss += loss.item()
    
    scheduler.step()
    return epoch_loss / len(dataloader)

def validate(model, dataloader, device):
    """
    Runs validation, finds the best threshold, and returns the best F0.5 score,
    threshold, precision, and recall.
    """
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for images, masks in tqdm(dataloader, desc="Validating"):
            images = images.to(device)
            outputs = model(images)
            all_preds.append(outputs.cpu().sigmoid())
            all_targets.append(masks.cpu())
            
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets).bool() # Ensure targets are boolean for efficient masking

    best_score = 0
    best_threshold = 0.5
    best_precision = 0
    best_recall = 0
    
    thresholds = np.arange(0.2, 0.8, 0.05)
    for threshold in thresholds:
        preds_binary = (all_preds > threshold)
        
        tp = (preds_binary & all_targets).sum().item()
        fp = (preds_binary & ~all_targets).sum().item()
        fn = (~preds_binary & all_targets).sum().item()
        
        score, precision, recall = calc_fbeta(tp, fp, fn, beta=0.5)
        
        if score > best_score:
            best_score = score
            best_threshold = threshold
            best_precision = precision
            best_recall = recall
            
    return best_score, best_threshold, best_precision, best_recall

print("Training and validation helper functions defined (with threshold search).")

Training and validation helper functions defined (with threshold search).


In [18]:
# --- 8. Main Training Loop ---

scaler = GradScaler()
best_val_score = 0
best_epoch = -1
epochs_no_improve = 0
model_save_path = f'best_model_fold_{CFG.VALID_FRAGMENTS[0]}.pth'

print("Starting training with improved validation...")

for epoch in range(1, CFG.EPOCHS + 1):
    start_time = time.time()
    
    train_loss = train_one_epoch(model, optimizer, scheduler, train_loader, CFG.DEVICE, scaler)
    val_score, best_threshold, precision, recall = validate(model, valid_loader, CFG.DEVICE)
    
    elapsed_time = time.time() - start_time
    
    print(f"Epoch {epoch}/{CFG.EPOCHS} - Time: {elapsed_time:.0f}s - Train Loss: {train_loss:.4f} - Val F0.5: {val_score:.4f} @ thr={best_threshold:.2f} (P: {precision:.4f}, R: {recall:.4f})")
    
    if val_score > best_val_score:
        best_val_score = val_score
        best_epoch = epoch
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_save_path)
        print(f"  >> New best score! Model saved to {model_save_path}")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= CFG.PATIENCE:
            print(f"\nEarly stopping triggered after {CFG.PATIENCE} epochs with no improvement.")
            break

print(f"\nTraining finished. Best validation F0.5 score: {best_val_score:.4f} at epoch {best_epoch}")

/tmp/ipykernel_699/2358857279.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Starting training with improved validation...


Training:   0%|          | 0/35 [00:00<?, ?it/s]

/tmp/ipykernel_699/1049546356.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Training:   3%|▎         | 1/35 [00:01<00:50,  1.50s/it]

...

Validating:  97%|█████████▋| 116/120 [00:18<00:00,  6.62it/s]

Validating:  98%|█████████▊| 117/120 [00:18<00:00,  6.63it/s]

Validating:  98%|█████████▊| 118/120 [00:18<00:00,  6.63it/s]

Validating:  99%|█████████▉| 119/120 [00:18<00:00,  6.64it/s]

Validating: 100%|██████████| 120/120 [00:18<00:00,  6.35it/s]